# Machine Learning Engineer Nanodegree
## Capstone Project - Who Wrote This? 
... or how to use natural languange processing to identify the author of anonymous articles.

Stefan Dittforth  
February 27th, 2018

--------------

## Sources:

[1] ["Who’s At The Keyboard? Authorship Attribution in Digital Evidence Investigations"](www.utica.edu/academic/institutes/ecii/publications/articles/B49F9C4A-0362-765C-6A235CB8ABDFACFF.pdf): good reference regarding number of authors and text volume.

[2] ["How a Computer Program Helped Show J.K. Rowling write A Cuckoo’s Calling"](https://www.scientificamerican.com/article/how-a-computer-program-helped-show-jk-rowling-write-a-cuckoos-calling/): some good ideas for feature to analyse.

[3] [How to solve 90% of NLP problems: a step-by-step guide](https://blog.insightdatascience.com/how-to-solve-90-of-nlp-problems-a-step-by-step-guide-fda605278e4e)

## Research Notes:

- test approaches suggested in [1]
- measure average word lentgh per article, distribution of word lengths, the 100 most common words, distribution of character 4-grams, word bigrams
- test with support vector machines
- using word embeddings as a much rich representation
Some good tips from [3]:
- Clean data:
    - Remove all irrelevant characters such as any non alphanumeric characters
    - Tokenize your text by separating it into individual words
    - Remove words that are not relevant, such as “@” twitter mentions or urls
    - Convert all characters to lowercase, in order to treat words such as “hello”, “Hello”, and “HELLO” the same
    - Consider combining misspelled or alternately spelled words to a single representation (e.g. “cool”/”kewl”/”cooool”)
    - Consider lemmatization (reduce words such as “am”, “are”, and “is” to a common form such as “be”)
- do random guess model
- One-hot encoding (Bag of Words)
- do we do bag-of-words one sentence level or article level? hmm ...
- apply PCA to visualise bag of words
- start with testing logistic regression for classification
- visualise confusion matrix
- vocabulary structure TF-IDF
- Word2Vec?
- LSTM
- Convolutional Neural Networks for Sentence Classification





----------------
## Introduction

write some introduction

----------------
## Data Acquisition

In order to allow the system to learn the writing characteristics of different authors we require a dataset that provides a large number of articles for individual authors. There are rich datasets for NLP research available in the public domain. A list, as an example, can be found [here](https://github.com/niderhoff/nlp-datasets). However, as part of this project we will build our own dataset. We will develop a web scraper that will collect articles from the publishing platform Medium. The articles on Medium seem to be reasonably long (at least several hundred words). There are enough authors that have published several hundreds articles. With this, it appears feasible to acquire a large enough data set to learn patterns in the writing characteristics to distinguish between individual authors.

This approach has been chosen as an opportunity to develop practical experience not only in machine learning but also around data acquisition. In data science and machine learning the acquisition and preparation of high quality data is often the bigger challenge than the actual development of the machine learning system itself. In "[Datasets Over Algorithms](https://www.edge.org/response-detail/26587)" author Alexander Wissner-Gross notes that 

>*"the average elapsed time between key [machine learning] algorithm proposals and corresponding advances was about eighteen years, whereas the average elapsed time between key dataset availabilities and corresponding advances was less than three years, or about six times faster, suggesting that datasets might have been limiting factors in the advances."*.

Conveniently the website [Top Authors](https://topauthors.xyz/) has published a list of 300+ top Medium authors. The project folder contains the short script `get_list_of_Medium_authors.py` that has been used to extract the Medium URL for each author. The initial list of 300+ authors has been reduced to 25. The criteria for this reduction was the number of published articles. For the 25 authors there are at least 300 articles available. The Medium URLs for these authors can be found in file `Medium_authors_25.txt`.

<img src="notebook/Top Authors.jpg" alt="Drawing" style="width: 400px;"/>
<center>*getting a list of Medium 
authors*</center>

The actual collection of the articles is done with the script `pull_Medium_articles.py`. The script performs two steps. First, it builds a list of all article URLs and for each article saves author URL and article URL in JSON format in the file `Medium_article_urls.json`. Below is an example how the entries for three articles look like.

```javascript
{"author_URL": "https://medium.com/@tedr/latest\n",
 "article_URL": "https://medium.com/season-of-the-witch/etiquette-and-the-cancer-patient-630a50047448?source=user_profile---------1----------------"}
{"author_URL": "https://medium.com/@esterbloom/latest\n",
 "article_URL": "https://medium.com/the-billfold/failing-at-shoplifting-life-with-kesha-bc2600b1f440?source=user_profile---------789----------------"}
{"author_URL": "https://medium.com/@gassee/latest",
 "article_URL": "https://mondaynote.com/the-ny-times-un-free-at-last-df2eddba360b?source=user_profile---------281----------------"}
```

The second part performs the actual download of the articles. The script reads the article URL saved in `Medium_article_urls.json`, navigates to the website and reads the text information from the html code of the article website. Each article is saved in text format in its own file. For each author a folder is generated that contains the articles for that author. Initially it was intended to store all articles in JSON format in one file. This turned out to be very cumbersome when troubleshooting the `pull_Medium_articles.py` script. Having a folder structure that allows to do quick visual inspections over the list of files in a file manager proved very helpful. In addition, the smaller article files made it easier to spot check the downloaded text information in a text editor.

During research for this project several Python libraries for interacting with websites have been explored: [mechanize](https://pypi.python.org/pypi/mechanize/0.3.6), [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#), [scrapy](https://scrapy.org/) and the [Selenium WebDriver](https://www.seleniumhq.org/projects/webdriver/). Eventually the decision was made to use the Selenium WebDriver. The key reason for this was: the Medium website uses a two step login process. The users provides its email address and then receives a one time login link via this email. That made it difficult to automate the login via script and ruled out all the libraries that don't allow user interaction with the website.

Once an article website is loaded, the required information can be pulled from the text attribute of specific html elements. The code snipped below shows the commands used to get the author name and the article text.

```python
author = self.browser.find_element_by_xpath('//a[@rel="author cc:attributionUrl"]').text
body = self.browser.find_element_by_xpath('//div[@class="postArticle-content js-postField js-notesSource js-trackedPost"]').text
```

As shown in the code snippet above the right elements are addressed by their respective xpath. Finding these xpaths required a bit of trail and error. A valuable tool for this is the FireFox Inspector. It allows to inspect the code and structure of a website and to find the right path to the right html element.

![Top Authors](notebook\Firefox Inspector.jpg)
<center>*finding the right xpath with Firefox Inspector*</center>

After the `pull_Medium_articles.py` script completed, the folder `Medium_articles` containing all article files has been compressed into a ZIP archive to preserve storage. With [`zipfile`](https://docs.python.org/3/library/zipfile.html) Python provides a library to work with ZIP archives. Going forward in this Notebook we will make use of this library to work with the files directly within the ZIP archive without the need to extract the archive.

Developing a web scraper script poses its own challenges. The initial idea is pretty straightforward: here is a list of URLs, go to each website, download the text part and save it in a file. As always, the pitfalls are discovered during implementation. Some time had to be invested to understand the structure of the Medium article websites and figure out the best way to find the right html elements that contain the required information. The Selenium WebDriver is not the most effective tool when it comes to scraping several thousand websites. The time to render each and every website adds up. An attempt has been made by parallelising the article download with multi-threading and spawning of several instances of the Firefox browser. This failed. It turned out that the fast sequence of websites caused Firefox to slowly consume all available memory and eventually Firefox stopped fetching new websites. In a parallelised version of the script the problem was only exaggerated. Finally, a pragamatic approach was taken and the script has been amended with the capability to continue the work where it has left off from a previous run. Over the course of several days the script has been restarted several times and eventually saved all articles.

In defense for Selenium, it needs to be noted that Selenium first and foremost is a tool to automate testing of websites and not a tool for scaping several thousand websites. The primary goal behind the `pull_Medium_articles.py` script was to get the data for this capstone project and not to develop a sophisticated web scraper. In this respect Selenium did the job. Despite the challenges, developing the web scraper script has been a worthwhile learing experience. It provided an opportunity to develop practical experience not only in machine learning but also around data acquisition.

In the next section we will explore the data set. We will also check what data preparation or cleaning activities might be required before we can apply algorithms  to the data set.

----------------
## Exploring the Data Set

All articles have been downloaded in individual text files and into folders for each author. This folder and file structure has been archived into `Medium_articles.zip`. Let's go through the files and get an overview about the data set. 

In [4]:
import zipfile
import json
import pandas as pd

def get_next_article_from_zip_file(file_name):
    """
    A generator function that reads the files in a zip archive one by one.
    """
    zf = zipfile.ZipFile(file_name, 'r')
    names = zf.namelist()
    for name in names:
        if name.endswith('.json'):
            data = zf.read(name)
            yield data
    zf.close()
    return

# read all articles into a pandas data frame
articles = pd.DataFrame(columns=['url', 'author', 'headline', 'body'])
for article_file in get_next_article_from_zip_file('Medium_articles.zip'):
    article = json.loads(article_file)
    articles = articles.append(json.loads(article_file), ignore_index=True)

In [5]:
# show a list of authors and the number of articles for each author
summary = articles[['author', 'body']].groupby('author').count()
summary.columns = ['number of articles']
display(summary.sort_values('number of articles', ascending=False))
print('Total number of articles: {:,}'.format(len(articles)))

number of articles
author                               
Nicole Dieker                    1948
Fabricio Teixeira                1733
Ester Bloom                      1328
Ethan Siegel                     1323
Yann Girard                      1042
Pascal Finette                   1020
Jon Westenberg 🌈                  932
Chris Messina                     872
Mark Suster                       825
howardlindzon                     795
umair haque                       688
James Altucher                    680
Johnson Kee                       660
Larry Kim                         619
Sean Everett                      558
Tim Boucher                       451
M.G. Siegler                      377
Nir Eyal                          366
Srinivas Rao                      355
Brad Feld                         350
Todd Brison                       340
Thaddeus Howze                    339
Jean-Louis Gassée                 337
Ted Rheingold                     319
Gary Vaynerchuk                   307

Total number of articles: 18,564


## Cleaning the Dataset

The articles were downloaded by extracting the 'text' attribute from html elements that contain the body text of the article. Many articles contain images, URLs to other pages, etc. We are interested to see if the text extracts still contain html fragments we might need to clean out. We do a qick check by listing the articles that contain the '<' and '>' characters. Those two characters enclose HTML tags. 

In [6]:
import re

regex = r'<.*>' # find any HTML tag enclosing brackets
counter = 0
for index, article in articles.iterrows():
    matches = re.finditer(regex, article['body'], re.DOTALL)
    for matchNum, match in enumerate(matches):
        counter = counter + 1
        print('Match number: {}'.format(counter))
        print('Match in article index: {}'.format(index))
        print(article['url'])
        print(match.group())

Match number: 1
Match in article index: 239
https://medium.com/@bfeld/people-are-the-price-of-admission-7782d2c55bba?source=user_profile---------85----------------
<riff>
We have a set of filters. For an early stage investment, we only invest in our themes. We only invest in the US. We don’t have to be the first money in a company, but if the company has raised more than $5m, it’s too late for us. Our goal with this filter is to say no to almost everything within 60 seconds.
Assuming something passes through this filter, we then focus on three things.
Do we have an affinity for the product? We don’t have to be daily users of the product, but we have to care about it in some way.
Are the founders obsessed (not passionate, but obsessed) about what they are building? Passion is easy to fake. Obsession is not.
Do the founders want us to be investors in their company as much as we want to be investors in their company? If it’s not bi-directional, that’s fine, but it’s not for us.
</riff>
Ma

There are only 156 occurences for the '<' and '>' characters in over 18,500 articles. Doing some spot checks it appears that the tag brackets are genuine parts of the article text (for example the text is about HTML coding). We will leave them in for now. No need to clean them out.

Let's have a look at some article texts.

In [7]:
import random

print('A few example articles:')
for _ in range(1,4):
    i = random.randrange(len(articles))
    print('----------------------')
    print('Index: {}'.format(i))
    print('URL: {}'.format(articles.iloc[i]['url']))
    print('Author: {}'.format(articles.iloc[i]['author']))
    print('Headline: {}'.format(articles.iloc[i]['headline']))
    print('Body: {}'.format(articles.iloc[i]['body']))

A few example articles:
----------------------
Index: 13560
URL: https://read.theheretic.org/clear-concise-communication-is-key-69b01bfb7bff?source=user_profile---------88----------------
Author: Pascal Finette
Headline: Clear, Concise Communication is Key
Body: Clear, Concise Communication is Key
It is pretty sad to report that all too many companies I meet struggle with clearly expressing what they are doing, how it works and why it matters. All too often I find myself in conversations where I have to ask: What is it exactly that you are doing? How does this work? Which problem does this solve again?
Sometimes I question my own intellect — but then I see other people react with the same confused look on their faces.
I can not stress enough how paramount it is that you can formulate a clear, concise, non-convoluted explanation of your company and what you’re doing. You don’t get points for turning your pitch into a scientific paper or a beautiful play on language. Nor does anyone give

The code above randomly selects 3 articles. We run it a few times. At each run we spot check the text in the 'Body' section. It appears that the text is clean and good to go for our next step.

Typically NLP text cleaning tasks include removing punctuation characters. For now we will keep them. In one of our first classification approaches we will use them to engineer features around number and length of sentences and paragraphs in articles.

---
## Balancing the Data Set

As seen above in the table with the number of articles per authors the data set is skewed. The number of articles ranges from 1,948 for Nicole Dieker and 307 for Gary Vaynerchuk. To avoid that our system develops a bias towards authors with a high number of articles we will balance the data set. This will be done by keeping the number of articles for each author equal to the author with the lowest number of articles.

In [8]:
# get the smallest number for articles for one author 
num_articles_to_keep = summary.min(axis=0)['number of articles']

# only keep the smallest number of articles for each author
indices_to_keep = []
for author, num_of_articles in summary.iterrows():
    indices_to_keep = indices_to_keep + list(articles[articles.author == author][:num_articles_to_keep].index.values)
articles = articles.iloc[indices_to_keep]

# check number of articles for each author
summary = articles[['author', 'body']].groupby('author').count()
summary.columns = ['number of articles']
display(summary.sort_values('number of articles', ascending=False))
print('Total number of articles: {:,}'.format(len(articles)))

number of articles
author                               
Brad Feld                         307
Nicole Dieker                     307
howardlindzon                     307
Yann Girard                       307
Todd Brison                       307
Tim Boucher                       307
Thaddeus Howze                    307
Ted Rheingold                     307
Srinivas Rao                      307
Sean Everett                      307
Pascal Finette                    307
Nir Eyal                          307
Mark Suster                       307
Chris Messina                     307
M.G. Siegler                      307
Larry Kim                         307
Jon Westenberg 🌈                  307
Johnson Kee                       307
Jean-Louis Gassée                 307
James Altucher                    307
Gary Vaynerchuk                   307
Fabricio Teixeira                 307
Ethan Siegel                      307
Ester Bloom                       307
umair haque                       307

Total number of articles: 7,675


## Model 1: The Baseline Model - Just Guess the Author



## Model 2: Basic Article Metrics

Let's first come up with some simple metrics that might help us to predict the author of an article.  Some metrics that come to mind are: the total, mean, median, min and max number of words in paragraphs, sentences and the article itself. When looking at different authors it appears that these features might be useful differentiators. Some authors have a tendency to longer articles. Others use longer sentences or shorter paragraphs.

In [82]:
import re
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pprint

pp = pprint.PrettyPrinter(indent=2)

def tokenize_paragraphs(text):
    paragraphs = [p for p in text.split('\n')]
    paragraphs_tokenized = []
    for paragraph in paragraphs:
        sentences = tokenize_sentences(paragraph)
        paragraphs_tokenized.append(sentences)
    num_paragraphs = len(paragraphs_tokenized)
    text_tokenized = {'num_paragraphs': num_paragraphs,
                      'paragraphs': paragraphs_tokenized}
    return text_tokenized

def tokenize_sentences(paragraph):
    sentences = sent_tokenize(paragraph)
    sentences_tokenized = []
    for sentence in sentences:
        words = tokenize_words(sentence)
        sentences_tokenized.append(words)
    num_sentences = len(sentences_tokenized)
    paragraph_tokenized = {'num_sentences': num_sentences,
                           'sentences': sentences_tokenized}
    return paragraph_tokenized

def tokenize_words(sentence):
    words_tokenized = word_tokenize(sentence)
    # remove punctuations from word list
    words_tokenized = [w for w in words_tokenized 
                       if not re.fullmatch('[' + string.punctuation + '’“‘”–…' ']', w)]
    num_words = len(words_tokenized)
    sentence_tokenized = {'num_words': num_words,
                          'words': words_tokenized}
    return sentence_tokenized

def get_article_metrics(body):
    body_tokenized = tokenize_paragraphs(body)
    paragraphs_sentences = []
    sentences_words = []
    for paragraph in body_tokenized['paragraphs']:
        paragraphs_sentences.append(paragraph['num_sentences'])
        for sentence in paragraph['sentences']:
            sentences_words.append(sentence['num_words'])
    paragraphs_sentences = pd.Series(paragraphs_sentences)
    sentences_words = pd.Series(sentences_words)
    metrics = {'article_num_paragraphs': body_tokenized['num_paragraphs'],
               'article_num_sentences': paragraphs_sentences.sum(),
               'article_num_words': sentences_words.sum(),
               'paragraphs_min_sentences': paragraphs_sentences.min(),
               'paragraphs_max_sentences': paragraphs_sentences.max(),
               'paragraphs_mean_sentences': paragraphs_sentences.mean(),
               'paragraphs_median_sentences': paragraphs_sentences.median(),
               'sentences_min_words': sentences_words.min(),
               'sentences_max_words': sentences_words.max(),
               'sentences_mean_words': sentences_words.mean(),
               'sentences_median_words': sentences_words.median()}
    return metrics

# calculate the basic metrics for each article
articles_metrics = pd.DataFrame(list(articles['body'].map(get_article_metrics)),
                                index=articles.index)
articles_metrics['author'] = articles['author']
display(articles_metrics)

article_num_paragraphs  article_num_sentences  article_num_words  \
0                           3                      5                 67   
1                           3                      4                 68   
2                          13                     45                708   
3                           5                     10                 90   
4                          16                     40                582   
5                           7                     23                424   
6                          19                     52                860   
7                           6                     12                223   
8                          14                     42                755   
9                          14                     32                494   
10                         16                     41                693   
11                          5                      6                 99   
12                          5                      7                109   
13                          5                      7                109   
14                          6                     12                138   
15                         12                     14                150   
16                          9                     17                277   
17                          7                      6                 20   
18                          3                      3                 30   
19                         11                     28                449   
20                          9                     14                240   
21                         12                     28                413   
22                         12                     30                495   
23                         11                     26                491   
24                          3                      5                 77   
25                         11                     21                445   
26                         11                     28                447   
27                         11                     39                509   
28                          7                     23                274   
29                         10                     35                685   
...                       ...                    ...                ...   
17111                      21                    130               1585   
17112                      26                     93                830   
17113                      25                     86                640   
17114                      11                     40                869   
17115                      25                    112               1210   
17116                      28                    144               1406   
17117                     169                    432               4496   
17118                      25                     97                976   
17119                      13                     56               1357   
17120                      24                     60                559   
17121                      18                     48                471   
17122                      24                     77                548   
17123                      24                     90                798   
17124                      27                     78                790   
17125                      32                    125               1327   
17126                      24                    100               1023   
17127                      15                     99                769   
17128                      24                    109               1099   
17129                      24                    119               1002   
17130                      19                    112               1261   
17131                      17                     68                861   
17132                      17   

In [ ]:
# display an example for a tokenized article
sample_article_index = 17125
body_tokenized = tokenize_paragraphs(articles.loc[sample_article_index]['body'])
print('\nSample article tokenized:\n')
pp.pprint(body_tokenized)

add some scatter plots to see if features allow separation

In [89]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# split labels into separate data frame
data = articles_metrics
labels = data[['author']]
data = data.drop('author', 1)
# shuffle training test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=23)
# decision tree prediction model
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
score_train = accuracy_score(y_train, clf.predict(X_train))
score_test = accuracy_score(y_test, clf.predict(X_test))
print('prediction accuracy score on training data set: {0}'.format(score_train))
print('prediction accuracy score on test data set: {0}'.format(score_test))b

prediction accuracy score on training data set: 0.9998262682418346
prediction accuracy score on test data set: 0.33559145388223033


display a confusion matrix